# 제주도 음식물 쓰레기 배출량 시계열 예측(Time Series Forecasting)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

필요한 packages, library 설치

In [ ]:
!pip install tzlocal
import tzlocal
!pip install rpy2
import rpy2
!pip install pmdarima==1.5.3
import pmdarima
import statsmodels
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

     |████████████████████████████████| 1.5 MB 8.9 MB/s 


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from rpy2.robjects.packages import importr

utils = importr('utils')
utils.install_packages('forecast')

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘xts’, ‘TTR’, ‘quadprog’, ‘quantmod’, ‘fracdiff’, ‘lmtest’, ‘timeDate’, ‘tseries’, ‘urca’, ‘zoo’, ‘RcppArmadillo’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/xts_0.12.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 517875 bytes (505 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =


<rpy2.rinterface_lib.sexp.NULLType object at 0x7f7c51dbb4b0> [RTYPES.NILSXP]

# 01_음식물쓰레기_FOOD_WASTE_210811_update.csv 제공데이터를 전처리한 파일 불러오기

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/세미2차/제주도배출량예측/시계열용_음식물쓰레기_전처리.csv', encoding='cp949')
data

,base_date,emd_nm,region,emd_cd,em_cnt,em_g
0,2018-01-01,남원읍,서귀포시_동,50130253.0,304.0,1239600.0
1,2018-01-01,성산읍,서귀포시_동,50130259.0,90.0,424100.0
2,2018-01-01,표선면,서귀포시_동,50130320.0,112.0,452850.0
3,2018-01-01,대정읍,서귀포시_서,50130250.0,309.0,1264950.0
4,2018-01-01,안덕면,서귀포시_서,50130310.0,77.0,296200.0
...,...,...,...,...,...,...
50879,2021-06-26,알수없음,알수없음,알수없음,11.0,57850.0
50880,2021-06-27,알수없음,알수없음,알수없음,10.0,48450.0
50881,2021-06-28,알수없음,알수없음,알수없음,13.0,63050.0
50882,2021-06-29,알수없음,알수없음,알수없음,10.0,63400.0


In [ ]:
# data type 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50884 entries, 0 to 50883
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   base_date  50884 non-null  object 
 1   emd_nm     50884 non-null  object 
 2   region     50884 non-null  object 
 3   emd_cd     50884 non-null  object 
 4   em_cnt     50884 non-null  float64
 5   em_g       50884 non-null  float64
dtypes: float64(2), object(4)
memory usage: 2.3+ MB


# 시계열 모델 성능 파악을 위해 train, test data set 생성
- 기존 21년 6월까지 존재하는 기간에서, 예측 기간 2개월에 맞춰서 가장 최근 5, 6월을 제외한 21년 4월까지 데이터를 train data로 설정

In [ ]:
train = data.copy()

In [ ]:
train['base_date'] = pd.to_datetime(train['base_date'],format="%Y-%m-%d")

In [ ]:
train

,base_date,emd_nm,region,emd_cd,em_cnt,em_g
0,2018-01-01,남원읍,서귀포시_동,50130253.0,304.0,1239600.0
1,2018-01-01,성산읍,서귀포시_동,50130259.0,90.0,424100.0
2,2018-01-01,표선면,서귀포시_동,50130320.0,112.0,452850.0
3,2018-01-01,대정읍,서귀포시_서,50130250.0,309.0,1264950.0
4,2018-01-01,안덕면,서귀포시_서,50130310.0,77.0,296200.0
...,...,...,...,...,...,...
50879,2021-06-26,알수없음,알수없음,알수없음,11.0,57850.0
50880,2021-06-27,알수없음,알수없음,알수없음,10.0,48450.0
50881,2021-06-28,알수없음,알수없음,알수없음,13.0,63050.0
50882,2021-06-29,알수없음,알수없음,알수없음,10.0,63400.0


- test data는 5월, 6월 따로 생성

In [ ]:
test5 = train[(train['base_date'] >= '2021-05-01') & (train['base_date'] <= '2021-05-31')]
test6 = train[(train['base_date'] >= '2021-06-01') & (train['base_date'] <= '2021-06-30')]

In [ ]:
# test 생성 후 train 시계열 기간 설정
train = train[train['base_date'] < '2021-05-01']
train

,base_date,emd_nm,region,emd_cd,em_cnt,em_g
0,2018-01-01,남원읍,서귀포시_동,50130253.0,304.0,1239600.0
1,2018-01-01,성산읍,서귀포시_동,50130259.0,90.0,424100.0
2,2018-01-01,표선면,서귀포시_동,50130320.0,112.0,452850.0
3,2018-01-01,대정읍,서귀포시_서,50130250.0,309.0,1264950.0
4,2018-01-01,안덕면,서귀포시_서,50130310.0,77.0,296200.0
...,...,...,...,...,...,...
50818,2021-04-26,알수없음,알수없음,알수없음,11.0,71150.0
50819,2021-04-27,알수없음,알수없음,알수없음,5.0,58900.0
50820,2021-04-28,알수없음,알수없음,알수없음,4.0,47250.0
50821,2021-04-29,알수없음,알수없음,알수없음,10.0,59600.0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48334 entries, 0 to 50822
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   base_date  48334 non-null  datetime64[ns]
 1   emd_nm     48334 non-null  object        
 2   region     48334 non-null  object        
 3   emd_cd     48334 non-null  object        
 4   em_cnt     48334 non-null  float64       
 5   em_g       48334 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 2.6+ MB


# 지역별 월 단위 다운 샘플링 함수 정의

In [ ]:
def month_resampling(df):
    new_data = pd.DataFrame() 
    df['base_date'] = df['base_date'].astype(str)
    df['year_month'] = df['base_date'].str.slice(stop=7)
    year_month = df['year_month'].drop_duplicates() # 중복값 처리 = unique 고유값

    downsampling_data = df.groupby(['emd_nm', 'year_month']).em_g.sum()
    downsampling_data = pd.DataFrame(downsampling_data)
    downsampling_data = downsampling_data.reset_index(drop=False,inplace=False)
    
    for i in tqdm(df.emd_nm.unique()):
        dong = downsampling_data[downsampling_data['emd_nm']==i]

        dong = dong.merge(year_month,how='outer')

        dong = dong.sort_values(by=['year_month'], axis=0, ascending=True) 

        dong['emd_nm'] = dong['emd_nm'].fillna(i)
        
        new_data = pd.concat([new_data,dong],axis=0)
        
    return new_data

In [ ]:
test5 = month_resampling(test5)
test6 = month_resampling(test6)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
100%|██████████| 42/42 [00:00<00:00, 167.13it/s]


In [ ]:
test6

,emd_nm,year_month,em_g
0,남원읍,2021-06,62927050.0
0,성산읍,2021-06,85688760.0
0,표선면,2021-06,42453750.0
0,대정읍,2021-06,109850800.0
0,안덕면,2021-06,46467450.0
0,대륜동,2021-06,46745900.0
0,대천동,2021-06,56178400.0
0,동홍동,2021-06,86524400.0
0,서홍동,2021-06,29767600.0
0,송산동,2021-06,31243800.0


# train 데이터를 월 단위로 다운 샘플링

- 시계열 모델의 학습 데이터 전처리
- 7, 8월의 배출량을 예측하므로 월 단위로 설정

In [ ]:
resampling_data = month_resampling(train)
resampling_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
100%|██████████| 42/42 [00:00<00:00, 143.82it/s]


,emd_nm,year_month,em_g
0,남원읍,2018-01,42437700.0
1,남원읍,2018-02,37088050.0
2,남원읍,2018-03,42988350.0
3,남원읍,2018-04,42703500.0
4,남원읍,2018-05,42616750.0
...,...,...,...
35,알수없음,2020-12,2147470.0
36,알수없음,2021-01,2057700.0
37,알수없음,2021-02,1985100.0
38,알수없음,2021-03,2153800.0


In [ ]:
resampling_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1680 entries, 0 to 39
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   emd_nm      1680 non-null   object 
 1   year_month  1680 non-null   object 
 2   em_g        1592 non-null   float64
dtypes: float64(1), object(2)
memory usage: 52.5+ KB


# 연속형 변수 평가 척도 MAE 함수 생성

MAE = (1/n) * sum(abs(y - y_pred))

In [ ]:
def mae(df) :
  n = len(df['emd_nm'])
  df['dif'] = np.abs(df['em_g'] - df['em_g_pred'])
  result = sum(df['dif']) / n
  return result

# 시계열 모델링
- ARIMA, ETS, fbprophet 3개 적용 후 mae값 비교
- 예측 대상 변수에 로그정규화 후 모델링 적용하고 mae값 다시 비교

## 1. ARIMA 자기회귀누적이동평균모델

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

pandas2ri.activate()

auto_arima = """
    function(ts){
        library(stats)
        library(forecast)
        d_params = ndiffs(ts)
        model = auto.arima(ts, max.p=2, d=d_params)
        forecasted_data = forecast(model, h=2)
        out_df = data.frame(forecasted_data$mean)
        colnames(out_df) = c('amount')
        out_df
    }
"""

auto_arima = robjects.r(auto_arima)
ts = robjects.r('ts')
c = robjects.r('c')

final_pred_5 = []
final_pred_6 = []
dong = []

for i in tqdm(resampling_data.emd_nm.unique()):
    trash = resampling_data[resampling_data['emd_nm']==i]
    print(i)

    train = ts(trash['em_g'], frequency=12) 

    forecast = auto_arima(train)
    final_pred_5.append(float(forecast.values[0]))
    final_pred_6.append(float(forecast.values[1]))
    dong.append(i)

  0%|          | 0/42 [00:00<?, ?it/s]

남원읍


R[write to console]: Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

  7%|▋         | 3/42 [00:01<00:18,  2.13it/s]

성산읍
표선면
대정읍


 12%|█▏        | 5/42 [00:01<00:10,  3.68it/s]

...

조천읍
구좌읍


 95%|█████████▌| 40/42 [00:06<00:00,  6.93it/s]

한경면
한림읍


100%|██████████| 42/42 [00:06<00:00,  6.38it/s]

알수없음


### ARIMA의 5월 예측값-실제값 MAE

In [ ]:
dict_data5 = {'emd_nm':dong, 'em_g_pred':final_pred_5}
arima_5 = pd.DataFrame(dict_data5)

In [ ]:
arima_5 = arima_5.merge(test5, on='emd_nm') # mae 함수 계산 형식에 맞게 merge
arima_5

,emd_nm,em_g_pred,year_month,em_g
0,남원읍,6.195770e+07,2021-05,59816600.0
1,성산읍,6.352935e+07,2021-05,57482500.0
2,표선면,3.844470e+07,2021-05,36223050.0
3,대정읍,1.068184e+08,2021-05,113929800.0
4,안덕면,4.020925e+07,2021-05,44037200.0
5,대륜동,4.438090e+07,2021-05,46104350.0
6,대천동,5.370570e+07,2021-05,58341550.0
7,동홍동,9.923022e+07,2021-05,94027400.0
8,서홍동,4.011056e+07,2021-05,38265400.0
9,송산동,2.483454e+07,2021-05,26770600.0


In [ ]:
arima_5_mae = mae(arima_5)
arima_5_mae

7336526.8856250215

### ARIMA의 6월 예측값-실제값 MAE

In [ ]:
dict_data6 = {'emd_nm':dong, 'em_g_pred':final_pred_6}
arima_6 = pd.DataFrame(dict_data6)

In [ ]:
arima_6 = arima_6.merge(test6, on='emd_nm') # mae 함수 계산 형식에 맞게 merge
arima_6

,emd_nm,em_g_pred,year_month,em_g
0,남원읍,6.195770e+07,2021-06,62927050.0
1,성산읍,6.352935e+07,2021-06,85688760.0
2,표선면,3.844470e+07,2021-06,42453750.0
3,대정읍,1.085453e+08,2021-06,109850800.0
4,안덕면,4.020925e+07,2021-06,46467450.0
5,대륜동,4.438090e+07,2021-06,46745900.0
6,대천동,5.370570e+07,2021-06,56178400.0
7,동홍동,1.017525e+08,2021-06,86524400.0
8,서홍동,4.049484e+07,2021-06,29767600.0
9,송산동,2.514088e+07,2021-06,31243800.0


In [ ]:
arima_6_mae = mae(arima_6)
arima_6_mae

9140296.633400202

## 2. ETS 지수평활법

In [ ]:
import rpy2.robjects as robjects # r 함수를 파이썬에서 사용 가능하게 변환하는 모듈
from rpy2.robjects import pandas2ri # 파이썬 자료형과 R 자료형의 호환을 도와주는 모듈

# pandas2ri를 활성화 
pandas2ri.activate()

ets = """
    function(ts){
        library(forecast) # forecast 패키지 로드
        model = ets(ts) # AIC가 낮은 지수평활 모델을 찾음 
        forecasted_data = forecast(model, h=2) # 이후 2개월(h=2)을 예측
        out_df = data.frame(forecasted_data$mean) # 예측값을 R의 데이터프레임으로 변환
        colnames(out_df) = c('amount') # amount라는 열로 이름을 지정
        out_df
    }
"""
# r() 함수로 r 자료형을 파이썬에서 사용 가능
ets = robjects.r(ets)# str 형식으로 정의된 ets
ts = robjects.r('ts')# r 자료형 time series 자료형으로 만들어주는 함수
c = robjects.r('c') # r 자료형 벡터를 만들어주는 함수

final_pred_5 = []
final_pred_6 = []
dong = []

for i in tqdm(resampling_data.emd_nm.unique()):
    trash = resampling_data[resampling_data['emd_nm']==i]

    # R의 ts 함수로 time series 데이터로 변환
    train = ts(trash['em_g'], frequency=12) # 1년이 12개월
    # 지수평활법l
    forecast = ets(train)

    final_pred_5.append(float(forecast.values[0]))
    final_pred_6.append(float(forecast.values[1]))
    dong.append(i)

100%|██████████| 42/42 [00:00<00:00, 55.47it/s]


### ETS의 5월 예측값-실제값 MAE

In [ ]:
dict_data5 = {'emd_nm':dong, 'em_g_pred':final_pred_5}
ets_5 = pd.DataFrame(dict_data5)

In [ ]:
ets_5 = ets_5.merge(test5, on='emd_nm') # mae 함수 계산 형식에 맞게 merge
ets_5

,emd_nm,em_g_pred,year_month,em_g
0,남원읍,6.195776e+07,2021-05,59816600.0
1,성산읍,6.326779e+07,2021-05,57482500.0
2,표선면,3.835465e+07,2021-05,36223050.0
3,대정읍,1.063213e+08,2021-05,113929800.0
4,안덕면,4.019850e+07,2021-05,44037200.0
5,대륜동,4.438101e+07,2021-05,46104350.0
6,대천동,5.412919e+07,2021-05,58341550.0
7,동홍동,9.301214e+07,2021-05,94027400.0
8,서홍동,3.931467e+07,2021-05,38265400.0
9,송산동,2.242936e+07,2021-05,26770600.0


In [ ]:
ets_5_mae = mae(ets_5)
ets_5_mae

5030560.992868645

### ETS의 6월 예측값-실제값 MAE

In [ ]:
dict_data6 = {'emd_nm':dong, 'em_g_pred':final_pred_6}
ets_6 = pd.DataFrame(dict_data6)

In [ ]:
ets_6 = ets_6.merge(test6, on='emd_nm') # mae 함수 계산 형식에 맞게 merge
ets_6

,emd_nm,em_g_pred,year_month,em_g
0,남원읍,6.195776e+07,2021-06,62927050.0
1,성산읍,6.326779e+07,2021-06,85688760.0
2,표선면,3.835465e+07,2021-06,42453750.0
3,대정읍,1.083726e+08,2021-06,109850800.0
4,안덕면,4.019850e+07,2021-06,46467450.0
5,대륜동,4.438101e+07,2021-06,46745900.0
6,대천동,5.412919e+07,2021-06,56178400.0
7,동홍동,9.301214e+07,2021-06,86524400.0
8,서홍동,3.931467e+07,2021-06,29767600.0
9,송산동,2.242936e+07,2021-06,31243800.0


In [ ]:
ets_6_mae = mae(ets_6)
ets_6_mae

6794742.793197806

## fbprophet 예측

- fbprophet 시계열 알고리즘은 datetime 변수의 컬럼명, 예측 대상인 종속변수의 컬럼명을 지정해야 모델링이 가능

In [ ]:
data1 = data.copy()

In [ ]:
resampling_data_p = month_resampling(data1)
resampling_data_p

100%|██████████| 42/42 [00:00<00:00, 184.10it/s]


,emd_nm,year_month,em_g
0,남원읍,2018-01,42437700.0
1,남원읍,2018-02,37088050.0
2,남원읍,2018-03,42988350.0
3,남원읍,2018-04,42703500.0
4,남원읍,2018-05,42616750.0
...,...,...,...
37,알수없음,2021-02,1985100.0
38,알수없음,2021-03,2153800.0
39,알수없음,2021-04,1936900.0
40,알수없음,2021-05,1910750.0


In [ ]:
resampling_data_p['year_month']= pd.to_datetime(resampling_data_p['year_month'])
resampling_data_p=resampling_data_p.rename(columns = {'year_month' : 'ds', 'em_g':'y'})
resampling_data_p

,emd_nm,ds,y
0,남원읍,2018-01-01,42437700.0
1,남원읍,2018-02-01,37088050.0
2,남원읍,2018-03-01,42988350.0
3,남원읍,2018-04-01,42703500.0
4,남원읍,2018-05-01,42616750.0
...,...,...,...
37,알수없음,2021-02-01,1985100.0
38,알수없음,2021-03-01,2153800.0
39,알수없음,2021-04-01,1936900.0
40,알수없음,2021-05-01,1910750.0


In [ ]:
!python --version

Python 3.7.11


In [ ]:
!pip install pystan==2.19.1.1
!pip install prophet

     |████████████████████████████████| 65 kB 2.3 MB/s 
     |████████████████████████████████| 49 kB 5.0 MB/s 
     |████████████████████████████████| 178 kB 19.0 MB/s 
  Created wheel for prophet: filename=prophet-1.0.1-py3-none-any.whl size=6640973 sha256=0b5e3ca2f6e0bf2a2079dfb063d548151836e15bd34b095048b32f35f80625aa
  Stored in directory: /root/.cache/pip/wheels/4e/a0/1a/02c9ec9e3e9de6bdbb3d769d11992a6926889d71567d6b9b67
Successfully built prophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, but you have cmdstanpy 0.9.68 which is incompatible.


In [ ]:
import fbprophet
print(fbprophet.__version__)

0.7.1


In [ ]:
from fbprophet import Prophet

In [ ]:
final_pred_5 = []
final_pred_6 = []
dong = []

last_2month = list()
for i in range(1, 7):
    last_2month.append(['2021-%02d' % i])
last_2month = pd.DataFrame(last_2month, columns = ['ds'])
last_2month['ds']= pd.to_datetime(last_2month['ds'])


for i in tqdm(resampling_data_p.emd_nm.unique()):
    selected_train = resampling_data_p[resampling_data_p['emd_nm']==i]

    # 학습을 위해 마지막 6개월 제외시키기
    train = selected_train.drop(selected_train.index[-6:])
    y_true = selected_train['y'][-6:].values

    model = Prophet()
    model.daily_seasonality=True
    model.weekly_seasonality=True
    model.fit(train)

    forecast = model.predict(last_2month)
    y_pred = forecast['yhat'].values

    final_pred_5.append(float(y_pred[0]))
    final_pred_6.append(float(y_pred[1]))
    dong.append(i)

 88%|████████▊ | 37/42 [01:45<00:15,  3.11s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 10.
 90%|█████████ | 38/42 [01:47<00:11,  2.76s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 10.
 93%|█████████▎| 39/42 [01:48<00:07,  2.48s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 10.
 95%|█████████▌| 40/42 [01:50<00:04,  2.29s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 10.
100%|██████████| 42/42 [01:55<00:00,  2.75s/it]


### fbprophet의 5월 예측값-실제값 MAE

In [ ]:
dict_data5 = {'emd_nm':dong, 'em_g_pred':final_pred_5}
phet_5 = pd.DataFrame(dict_data5)

In [ ]:
phet_5 = phet_5.merge(test5, on='emd_nm') # mae 함수 계산 형식에 맞게 merge
phet_5

,emd_nm,em_g_pred,year_month,em_g
0,남원읍,7.431015e+07,2021-05,59816600.0
1,성산읍,7.988460e+07,2021-05,57482500.0
2,표선면,4.584629e+07,2021-05,36223050.0
3,대정읍,1.162574e+08,2021-05,113929800.0
4,안덕면,5.465468e+07,2021-05,44037200.0
5,대륜동,5.991919e+07,2021-05,46104350.0
6,대천동,6.599136e+07,2021-05,58341550.0
7,동홍동,1.365528e+08,2021-05,94027400.0
8,서홍동,3.999615e+07,2021-05,38265400.0
9,송산동,2.180098e+07,2021-05,26770600.0


In [ ]:
phet_5_mae = mae(phet_5)
phet_5_mae

9495053.315228876

### fbprophet의 6월 예측값-실제값 MAE

In [ ]:
dict_data6 = {'emd_nm':dong, 'em_g_pred':final_pred_6}
phet_6 = pd.DataFrame(dict_data6)

In [ ]:
phet_6 = phet_6.merge(test6, on='emd_nm') # mae 함수 계산 형식에 맞게 merge
phet_6

,emd_nm,em_g_pred,year_month,em_g
0,남원읍,3.026081e+07,2021-06,62927050.0
1,성산읍,1.164681e+08,2021-06,85688760.0
2,표선면,4.875503e+07,2021-06,42453750.0
3,대정읍,1.330218e+08,2021-06,109850800.0
4,안덕면,5.402738e+07,2021-06,46467450.0
5,대륜동,1.339388e+07,2021-06,46745900.0
6,대천동,3.874633e+07,2021-06,56178400.0
7,동홍동,2.885104e+07,2021-06,86524400.0
8,서홍동,1.013415e+07,2021-06,29767600.0
9,송산동,-2.586750e+06,2021-06,31243800.0


In [ ]:
phet_6_mae = mae(phet_6)
phet_6_mae

18549871.869553808

# 시계열 모델링 + 로그정규화 적용
- 각각의 모델로 예측한 후, 예측 대상인 음식물 쓰레기 배출량(em_g)에 log 정규화를 진행하여 다시 모델을 적용한다.

- g단위의 연속형 변수의 정규성을 높여, 데이터 간 편차를 줄일 수 있다.

- fbprophet의 경우, arima와 ets와 다르게 MAE값이 좋지 않다고 판단하여 로그 정규화에서 제외.

## 1. ARIMA + 로그정규화

In [ ]:
import rpy2.robjects as robjects # r 함수를 파이썬에서 사용 가능하게 변환하는 모듈
from rpy2.robjects import pandas2ri # 파이썬 자료형과 R 자료형의 호환을 도와주는 모듈
import numpy as np

# pandas2ri를 활성화 
pandas2ri.activate()

auto_arima = """
    function(ts){
        library(forecast) # forecast 패키지 로드
        d_params = ndiffs(ts) # 시계열 자료의 차분 횟수 계산
        model = auto.arima(ts, max.p=2, d=d_params) # auto.arima 모델 생성
        forecasted_data = forecast(model, h=2) # 이후 2개월(h=2)을 예측
        out_df = data.frame(forecasted_data$mean) # 예측값을 R의 데이터프레임으로 변환
        colnames(out_df) = c('amount') # amount라는 열로 이름을 지정
        out_df
    }
"""

# r() 함수로 r 자료형을 파이썬에서 사용 가능
auto_arima = robjects.r(auto_arima)
ts = robjects.r('ts')# r 자료형 time series 자료형으로 만들어주는 함수
c = robjects.r('c') # r 자료형 벡터를 만들어주는 함수
log = robjects.r('log')# 로그 변환 함수
exp = robjects.r('exp')# 로그 역변환 함수

final_pred_5 = []
final_pred_6 = []
dong = []

for i in tqdm(resampling_data.emd_nm.unique()):
    trash = resampling_data[resampling_data['emd_nm']==i]

    train = ts(log(trash['em_g']), frequency=12) 

    forecast = auto_arima(train)
    final_pred_5.append(float(exp(forecast.values[0])))
    final_pred_6.append(float(exp(forecast.values[1])))
    dong.append(i)

100%|██████████| 42/42 [00:05<00:00,  7.99it/s]


### ARIMA+log의 5월 예측값-실제값 MAE

In [ ]:
dict_data5 = {'emd_nm':dong, 'em_g_pred':final_pred_5}
arima_log_5 = pd.DataFrame(dict_data5)

In [ ]:
arima_log_5 = arima_log_5.merge(test5, on='emd_nm') # mae 함수 계산 형식에 맞게 merge
arima_log_5

,emd_nm,em_g_pred,year_month,em_g
0,남원읍,6.195770e+07,2021-05,59816600.0
1,성산읍,6.352935e+07,2021-05,57482500.0
2,표선면,3.844470e+07,2021-05,36223050.0
3,대정읍,1.082995e+08,2021-05,113929800.0
4,안덕면,4.020925e+07,2021-05,44037200.0
5,대륜동,4.574167e+07,2021-05,46104350.0
6,대천동,5.370570e+07,2021-05,58341550.0
7,동홍동,9.505794e+07,2021-05,94027400.0
8,서홍동,3.958101e+07,2021-05,38265400.0
9,송산동,2.356466e+07,2021-05,26770600.0


In [ ]:
arima_log_5_mae = mae(arima_log_5)
arima_log_5_mae

4913002.695682008

### ARIMA+log의 6월 예측값-실제값 MAE

In [ ]:
dict_data6 = {'emd_nm':dong, 'em_g_pred':final_pred_6}
arima_log_6 = pd.DataFrame(dict_data6)

In [ ]:
arima_log_6 = arima_log_6.merge(test6, on='emd_nm') # mae 함수 계산 형식에 맞게 merge
arima_log_6

,emd_nm,em_g_pred,year_month,em_g
0,남원읍,6.195770e+07,2021-06,62927050.0
1,성산읍,6.352935e+07,2021-06,85688760.0
2,표선면,3.844470e+07,2021-06,42453750.0
3,대정읍,1.111377e+08,2021-06,109850800.0
4,안덕면,4.020925e+07,2021-06,46467450.0
5,대륜동,4.674727e+07,2021-06,46745900.0
6,대천동,5.370570e+07,2021-06,56178400.0
7,동홍동,9.669010e+07,2021-06,86524400.0
8,서홍동,3.960340e+07,2021-06,29767600.0
9,송산동,2.396974e+07,2021-06,31243800.0


In [ ]:
arima_log_6_mae = mae(arima_log_6)
arima_log_6_mae

7023380.307934273

## 2. ETS + 로그정규화

In [ ]:
# 배출량 변동 계수를 구하는 함수
def coefficient_variation(df, i):
    cv_data = df.groupby(['emd_nm']).em_g.std()/df.groupby(['emd_nm']).em_g.mean()
    cv = cv_data[i]
    return cv

In [ ]:
import rpy2.robjects as robjects # r 함수를 파이썬에서 사용 가능하게 변환하는 모듈
from rpy2.robjects import pandas2ri # 파이썬 자료형과 R 자료형의 호환을 도와주는 모듈
import numpy as np

# pandas2ri를 활성화 
pandas2ri.activate()

ets = """
    function(ts){
        library(forecast) # forecast 패키지 로드
        model = ets(ts) # AIC가 낮은 지수평활 모델을 찾음 
        forecasted_data = forecast(model, h=2) # 이후 2개월(h=2)을 예측
        out_df = data.frame(forecasted_data$mean) # 예측값을 R의 데이터프레임으로 변환
        colnames(out_df) = c('amount') # amount라는 열로 이름을 지정
        out_df
    }
"""

# r() 함수로 r 자료형을 파이썬에서 사용 가능
ets = robjects.r(ets)
ts = robjects.r('ts') # r 자료형 time series 자료형으로 만들어주는 함수
c = robjects.r('c') # r 자료형 벡터를 만들어주는 함수
log = robjects.r('log') # 로그 변환 함수
exp = robjects.r('exp')# 로그 역변환 함수

final_pred_5 = []
final_pred_6 = []
dong = []

for i in tqdm(resampling_data.emd_nm.unique()):
    trash = resampling_data[resampling_data['emd_nm']==i]
    
    cv = coefficient_variation(resampling_data, i)
    # 배출량 변동 계수가 0.3 미만인 경우만 log를 씌움
    if cv < 0.3:
        train_log = ts(log(trash['em_g']), frequency=12) 
        # ets model
        forecast_log = ets(train_log)
        final_pred_5.append(float(exp(forecast_log.values[0])))
        final_pred_6.append(float(exp(forecast_log.values[1])))
        dong.append(i)
    # 배출량 변동 계수가 0.3 이상인 경우
    else:
        train = ts(trash['em_g'], frequency=12)
        # 지수평활법
        forecast = ets(train)
        final_pred_5.append(float(forecast.values[0]))
        final_pred_6.append(float(forecast.values[1]))
        dong.append(i)

100%|██████████| 42/42 [00:01<00:00, 33.67it/s]


### ETS+log의 5월 예측값-실제값 MAE

In [ ]:
dict_data5 = {'emd_nm':dong, 'em_g_pred':final_pred_5}
ets_log_5 = pd.DataFrame(dict_data5)

In [ ]:
ets_log_5 = ets_log_5.merge(test5, on='emd_nm') # mae 함수 계산 형식에 맞게 merge
ets_log_5

,emd_nm,em_g_pred,year_month,em_g
0,남원읍,6.195776e+07,2021-05,59816600.0
1,성산읍,6.326779e+07,2021-05,57482500.0
2,표선면,3.835465e+07,2021-05,36223050.0
3,대정읍,1.063213e+08,2021-05,113929800.0
4,안덕면,4.019850e+07,2021-05,44037200.0
5,대륜동,4.438101e+07,2021-05,46104350.0
6,대천동,5.370625e+07,2021-05,58341550.0
7,동홍동,9.301210e+07,2021-05,94027400.0
8,서홍동,3.931466e+07,2021-05,38265400.0
9,송산동,2.242935e+07,2021-05,26770600.0


In [ ]:
ets_log_5_mae = mae(ets_log_5)
ets_log_5_mae

4250173.175913585

### ETS+log의 6월 예측값-실제값 MAE

In [ ]:
dict_data6 = {'emd_nm':dong, 'em_g_pred':final_pred_6}
ets_log_6 = pd.DataFrame(dict_data6)

In [ ]:
ets_log_6 = ets_log_6.merge(test6, on='emd_nm') # mae 함수 계산 형식에 맞게 merge
ets_log_6

,emd_nm,em_g_pred,year_month,em_g
0,남원읍,6.195776e+07,2021-06,62927050.0
1,성산읍,6.326779e+07,2021-06,85688760.0
2,표선면,3.835465e+07,2021-06,42453750.0
3,대정읍,1.083726e+08,2021-06,109850800.0
4,안덕면,4.019850e+07,2021-06,46467450.0
5,대륜동,4.438101e+07,2021-06,46745900.0
6,대천동,5.370625e+07,2021-06,56178400.0
7,동홍동,9.301210e+07,2021-06,86524400.0
8,서홍동,3.931466e+07,2021-06,29767600.0
9,송산동,2.242935e+07,2021-06,31243800.0


In [ ]:
ets_log_6_mae = mae(ets_log_6)
ets_log_6_mae

5636556.796571074

# MAE값 오차 가장 낮은 ETS+log 시계열 모델 적용

## 전체 데이터를 월 단위로 다운 샘플링 적용

In [ ]:
resampling_data = month_resampling(data)
resampling_data

100%|██████████| 42/42 [00:00<00:00, 162.49it/s]


,emd_nm,year_month,em_g
0,남원읍,2018-01,42437700.0
1,남원읍,2018-02,37088050.0
2,남원읍,2018-03,42988350.0
3,남원읍,2018-04,42703500.0
4,남원읍,2018-05,42616750.0
...,...,...,...
37,알수없음,2021-02,1985100.0
38,알수없음,2021-03,2153800.0
39,알수없음,2021-04,1936900.0
40,알수없음,2021-05,1910750.0


In [ ]:
import rpy2.robjects as robjects # r 함수를 파이썬에서 사용 가능하게 변환하는 모듈
from rpy2.robjects import pandas2ri # 파이썬 자료형과 R 자료형의 호환을 도와주는 모듈
import numpy as np

# pandas2ri를 활성화 
pandas2ri.activate()

ets = """
    function(ts){
        library(forecast) # forecast 패키지 로드
        model = ets(ts) # AIC가 낮은 지수평활 모델을 찾음 
        forecasted_data = forecast(model, h=2) # 이후 2개월(h=2)을 예측
        out_df = data.frame(forecasted_data$mean) # 예측값을 R의 데이터프레임으로 변환
        colnames(out_df) = c('amount') # amount라는 열로 이름을 지정
        out_df
    }
"""

# r() 함수로 r 자료형을 파이썬에서 사용 가능
ets = robjects.r(ets)
ts = robjects.r('ts') # r 자료형 time series 자료형으로 만들어주는 함수
c = robjects.r('c') # r 자료형 벡터를 만들어주는 함수
log = robjects.r('log') # 로그 변환 함수
exp = robjects.r('exp')# 로그 역변환 함수

final_pred_7 = []
final_pred_8 = []
dong = []

for i in tqdm(resampling_data.emd_nm.unique()):
    trash = resampling_data[resampling_data['emd_nm']==i]
    
    cv = coefficient_variation(resampling_data, i)
    # 배출량 변동 계수가 0.3 미만인 경우만 log를 씌움
    if cv < 0.3:
        train_log = ts(log(trash['em_g']), frequency=12) 
        # ets model
        forecast_log = ets(train_log)
        final_pred_7.append(float(exp(forecast_log.values[0])))
        final_pred_8.append(float(exp(forecast_log.values[1])))
        dong.append(i)
    # 배출량 변동 계수가 0.3 이상인 경우
    else:
        train = ts(trash['em_g'], frequency=12)
        # 지수평활법
        forecast = ets(train)
        final_pred_7.append(float(forecast.values[0]))
        final_pred_8.append(float(forecast.values[1]))
        dong.append(i)

100%|██████████| 42/42 [00:00<00:00, 43.30it/s]


## 21년 7월 배출량 예측

In [ ]:
dict_data7 = {'emd_nm':dong, 'em_g_pred':final_pred_7}
model_7 = pd.DataFrame(dict_data7)
model_7

,emd_nm,em_g_pred
0,남원읍,6.292673e+07
1,성산읍,8.235627e+07
2,표선면,4.225807e+07
3,대정읍,1.131597e+08
4,안덕면,4.643683e+07
5,대륜동,4.674584e+07
6,대천동,5.617861e+07
7,동홍동,8.652512e+07
8,서홍동,2.976835e+07
9,송산동,3.124332e+07


## 21년 8월 배출량 예측

In [ ]:
dict_data8 = {'emd_nm':dong, 'em_g_pred':final_pred_8}
model_8 = pd.DataFrame(dict_data8)
model_8

,emd_nm,em_g_pred
0,남원읍,6.292673e+07
1,성산읍,8.235627e+07
2,표선면,4.225807e+07
3,대정읍,1.152269e+08
4,안덕면,4.643683e+07
5,대륜동,4.674584e+07
6,대천동,5.617861e+07
7,동홍동,8.652512e+07
8,서홍동,2.976835e+07
9,송산동,3.124332e+07


In [ ]:
dict_data = {'emd_nm':dong, 'em_g_pred_7':final_pred_7, 'em_g_pred_8':final_pred_8}
model_final = pd.DataFrame(dict_data)
model_final

,emd_nm,em_g_pred_7,em_g_pred_8
0,남원읍,6.292673e+07,6.292673e+07
1,성산읍,8.235627e+07,8.235627e+07
2,표선면,4.225807e+07,4.225807e+07
3,대정읍,1.131597e+08,1.152269e+08
4,안덕면,4.643683e+07,4.643683e+07
5,대륜동,4.674584e+07,4.674584e+07
6,대천동,5.617861e+07,5.617861e+07
7,동홍동,8.652512e+07,8.652512e+07
8,서홍동,2.976835e+07,2.976835e+07
9,송산동,3.124332e+07,3.124332e+07


# 최종 예측 결과 저장

In [ ]:
model_final.to_csv("/content/drive/MyDrive/세미2차/제주도배출량예측/제주배출량78월시계열예측결과.csv", index=False, encoding='CP949')